关联规则实例：银行金融产品推荐

In [12]:
#1.导入模块和数据
import pandas as pd
pd.options.display.width=200
pd.set_option('precision',4)
import os,sys
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori
from pandas import DataFrame
%matplotlib inline

In [74]:
#2.导入数据
contents=pd.read_csv('01-data-bank.csv')
contents.head(10)
#数据检查：计算唯一用户和产品数量
n_users=contents.ID.unique().shape[0]
n_items=contents.Product.unique().shape[0]
print('Number of total records='+str(len(contents)))
print('Number of users='+str(n_users)+' | Number of Product='+str(n_items))

Number of total records=35169
Number of users=7992 | Number of Product=14


In [32]:
#3.构建用户-产品  矩阵
basket_new=(contents.groupby(['ID','Product']).size().unstack().reset_index().fillna(0).set_index('ID')) #unstack()数据行转列
basket_new.iloc[0:10,:]

Product,ATM,AUTO,CCRD,CD,CKCRD,CKING,HMEQLC,IRA,MMDA,MTG,PLOAN,SVG,TRUST
ID,,,,,,,,,,,,,
500026.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
500075.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
500129.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
500256.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
500341.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
500350.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
500458.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
500595.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
500743.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0


In [78]:
#4.关联规则挖掘
#生成频繁项集合
frequent=apriori(basket_new,min_support=0.1,use_colnames=True,max_len=2)
frequent

,support,itemsets
0,0.3846,(ATM)
1,0.1548,(CCRD)
2,0.2453,(CD)
3,0.1130,(CKCRD)
4,0.8578,(CKING)
5,0.1647,(HMEQLC)
6,0.1084,(IRA)
7,0.1744,(MMDA)
8,0.6187,(SVG)
9,0.3619,"(CKING, ATM)"


In [75]:
#生成规则
rules=association_rules(frequent,metric='lift',min_threshold=0.8)
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(HMEQLC),(CKING),0.1647,0.8578,0.1647,1.0000,1.1657,0.0234,inf
1,(CKING),(HMEQLC),0.8578,0.1647,0.1647,0.1920,1.1657,0.0234,1.0338
2,(SVG),(CD),0.6187,0.2453,0.1572,0.2540,1.0358,0.0054,1.0118
3,(CD),(SVG),0.2453,0.6187,0.1572,0.6408,1.0358,0.0054,1.0616
4,(CCRD),(SVG),0.1548,0.6187,0.1022,0.6605,1.0675,0.0065,1.1230
5,(SVG),(CCRD),0.6187,0.1548,0.1022,0.1653,1.0675,0.0065,1.0125
6,(MMDA),(CKING),0.1744,0.8578,0.1558,0.8931,1.0411,0.0062,1.3300
7,(CKING),(MMDA),0.8578,0.1744,0.1558,0.1816,1.0411,0.0062,1.0088
8,(CKING),(SVG),0.8578,0.6187,0.5417,0.6315,1.0207,0.0110,1.0348
9,(SVG),(CKING),0.6187,0.8578,0.5417,0.8756,1.0207,0.0110,1.1428


In [48]:
#Step 5: 更改数据格式，添加筛选条件，输出结果
frame=DataFrame(rules)
R_result=frame[(frame['lift']>=1)&(frame['confidence']>=0.5)&(frame['antecedents']!=frozenset({'CD'}))&(frame['consequents']!=frozenset({'CD'}))]
R_result.sort_values(by=['lift'],axis=0,ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(HMEQLC),(CKING),0.1647,0.8578,0.1647,1.0000,1.1657,0.0234,inf
18,(CKCRD),(CKING),0.1130,0.8578,0.1130,1.0000,1.1657,0.0161,inf
13,(CCRD),(CKING),0.1548,0.8578,0.1485,0.9596,1.1186,0.0157,3.5170
17,(ATM),(CKING),0.3846,0.8578,0.3619,0.9411,1.0971,0.0320,2.4136
20,(HMEQLC),(SVG),0.1647,0.6187,0.1115,0.6771,1.0943,0.0096,1.1807
11,(ATM),(SVG),0.3846,0.6187,0.2569,0.6681,1.0798,0.0190,1.1488
4,(CCRD),(SVG),0.1548,0.6187,0.1022,0.6605,1.0675,0.0065,1.1230
6,(MMDA),(CKING),0.1744,0.8578,0.1558,0.8931,1.0411,0.0062,1.3300
8,(CKING),(SVG),0.8578,0.6187,0.5417,0.6315,1.0207,0.0110,1.0348
9,(SVG),(CKING),0.6187,0.8578,0.5417,0.8756,1.0207,0.0110,1.1428


In [49]:
#输出结果到本地
R_result.to_csv('01_R_reslut.csv', index=False, sep=',')